Probabilistic Retrival Model

In [5]:
# Install necessary libraries
!pip install python-terrier

In [6]:
# Import necessary libraries
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client

import nltk

import pandas as pd
import pyterrier as pt
import math

In [7]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [8]:
# Initialize PyTerrier
if not pt.started():
    pt.init()

In [9]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:r-lab-sose-2024/ir-acl-anthology-20240504-training')
# A (pre-built) PyTerrier index loaded from TIRA
# index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

In [10]:
corpus = [doc['abstract'] for doc in pt_dataset.get_corpus_iter() if 'abstract' in doc]

r-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:04<00:00, 25742.77it/s]


In [11]:
# Define the BM25 class
class BM25:
    def __init__(self, documents):
        self.documents = documents
        self.document_freq = {}
        self.avg_doc_length = 0
        self.k1 = 1.2
        self.b = 0.75
        self.calculate_document_freq()
        self.calculate_avg_doc_length()

    def calculate_document_freq(self):
        for doc in self.documents:
            for word in set(doc.split()):
                if word in self.document_freq:
                    self.document_freq[word] += 1
                else:
                    self.document_freq[word] = 1

    def calculate_avg_doc_length(self):
        total_length = sum(len(doc.split()) for doc in self.documents)
        self.avg_doc_length = total_length / len(self.documents)

    def calculate_score(self, query, document):
        score = 0
        query_terms = query.split()
        doc_terms = document.split()
        doc_length = len(doc_terms)

        for term in query_terms:
            if term in self.document_freq:
                term_freq = doc_terms.count(term)
                numerator = term_freq * (self.k1 + 1)
                denominator = term_freq + self.k1 * (1 - self.b + self.b * (doc_length / self.avg_doc_length))
                score += math.log((len(self.documents) - self.document_freq[term] + 0.5) / (self.document_freq[term] + 0.5)) * (numerator / denominator)

        return score


In [12]:
class BM25:
    def __init__(self, documents):
        self.documents = documents
        self.document_freq = {}
        self.avg_doc_length = 0
        self.k1 = 1.2
        self.b = 0.75
        self.calculate_document_freq()
        self.calculate_avg_doc_length()

    def calculate_document_freq(self):
        for doc in self.documents:
            for word in set(doc.split()):
                if word in self.document_freq:
                    self.document_freq[word] += 1
                else:
                    self.document_freq[word] = 1

    def calculate_avg_doc_length(self):
        total_length = sum(len(doc.split()) for doc in self.documents)
        self.avg_doc_length = total_length / len(self.documents)

    def calculate_score(self, query, document):
        score = 0
        query_terms = query.split()
        doc_terms = document.split()
        doc_length = len(doc_terms)

        for term in query_terms:
            if term in self.document_freq:
                term_freq = doc_terms.count(term)
                numerator = term_freq * (self.k1 + 1)
                denominator = term_freq + self.k1 * (1 - self.b + self.b * (doc_length / self.avg_doc_length))
                if denominator != 0:  # Check if denominator is zero
                    score += math.log((len(self.documents) - self.document_freq[term] + 0.5) / (self.document_freq[term] + 0.5)) * (numerator / denominator)

        return score


In [13]:
# Function to query the BM25 model
def query_bm25(query):
    scores = []
    for i, doc in enumerate(corpus):
        score = bm25.calculate_score(query, doc)
        scores.append((i, score))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return scores

In [14]:
# Sample query
query = "and"
results = query_bm25(query)

In [15]:
# Get the top 10 results
top_results = results[:10]

In [16]:
# Create a DataFrame to display the results
results_df = pd.DataFrame(top_results, columns=['Document Index', 'Score'])
results_df['Document'] = results_df['Document Index'].apply(lambda x: corpus[x])

In [17]:
# Display the results
results_df

,Document Index,Score,Document
